In [3]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [4]:
# === 入力 ===
df = pd.read_csv("jleague_manager_results.csv")

# === ネットワーク構築 ===
G = nx.Graph()

In [5]:
def club_converter(club_name):
    if club_name == "Ｆ東京":
        return "FC東京"
    elif club_name == "横浜Ｍ" or club_name == "横浜M":
        return "横浜FM"
    elif club_name == "市原":
        return "千葉"
    elif club_name == "V川崎" or club_name == "Ｖ川崎" or club_name == "Ｃ川崎" or club_name == "C川崎" or club_name == "東京V":
        return "東京Ｖ"
    elif club_name == "盛岡":
        return "岩手"
    elif club_name == "B仙台":
        return "仙台"
    elif club_name == "栃木":
        return "栃木SC"
    elif club_name == "平塚":
        return "湘南"
    elif club_name == "草津":
        return "群馬"
    else:
        return club_name

In [6]:
def check_special_cases(club, year):
    if "*1" in club:
        club = club.replace("*1", "")
        return club, str(year) + "（総監督）"
    elif "*2" in club:
        club = club.replace("*2", "")
        return club, str(year) + "（ヘッドコーチ）"
    elif "*3" in club:
        club = club.replace("*3", "")
        return club, str(year) + "（コーチ）"
    elif "*4" in club:
        club = club.replace("*4", "")
        return club, str(year) + "（GKコーチ）"
    else:
        return club, str(year)

In [10]:
import base64
import re
from math import cos, sin, pi
from pyvis.network import Network
import networkx as nx
import matplotlib.cm as cm
import matplotlib.colors as mcolors

G = nx.Graph()

def image_to_base64(path, default=None):
    """画像ファイルをBase64に変換。存在しない場合はdefaultを返す"""
    import os
    if not os.path.exists(path):
        if default:
            path = default
        else:
            return None
    with open(path, "rb") as f:
        return "data:image/png;base64," + base64.b64encode(f.read()).decode("utf-8")

# 年度に応じた色生成
def year_to_color(year):
    if year == 2025:
        return "#FF3333"  # 2025年は赤
    norm_year = (year - 1990) / (2025 - 1990)
    cmap = cm.Blues
    rgba = cmap(norm_year)
    return mcolors.to_hex(rgba)

# --- ノード・エッジ作成 ---
for _, row in df.iterrows():
    manager = row["監督名"]
    club = row["チーム"]
    year = str(row["年度"])

    if year == "-" or not year:
        continue
    # 数字以外を含む場合も最初の4桁を抽出
    m = re.search(r"\d{4}", year)
    if not m:
        continue
    year_int = int(m.group())
    if year_int < 1990:
        continue

    club, year_int = check_special_cases(club, year_int)
    club = club_converter(club)

    if not G.has_node(manager):
        G.add_node(manager, type="manager")
    if not G.has_node(club):
        G.add_node(club, type="club")

    if G.has_edge(manager, club):
        if not isinstance(G[manager][club]["years"], set):
            G[manager][club]["years"] = set([G[manager][club]["years"]])
        G[manager][club]["years"].add(str(year_int))
    else:
        G.add_edge(manager, club, years={str(year_int)})

# --- pyvis 可視化 ---
net = Network(height="750px", width="100%", bgcolor="#ffffff", directed=False, notebook=True)
net.toggle_hide_edges_on_drag(True)
net.toggle_hide_nodes_on_drag(True)

# --- クラブ座標を円形配置 ---
club_nodes = [n for n,d in G.nodes(data=True) if d.get("type")=="club"]
if not club_nodes:
    raise ValueError("クラブノードが存在しません")
angle_step = 2 * pi / len(club_nodes)
radius = 1800
club_positions = {}

for i, node in enumerate(club_nodes):
    x = radius * cos(i * angle_step)
    y = radius * sin(i * angle_step)
    club_positions[node] = (x, y)
    img_b64 = image_to_base64(f"images/{node}.png")
    club_default_color = "#1F77B4"   # 他の候補: "#4C9F70" (緑), "#8C8C8C" (グレー)
    if img_b64:
        net.add_node(node, label="", title=node, shape="image", image=img_b64, size=90, x=x, y=y, physics=False, group="club")
    else:
        net.add_node(node, label=node, title=node, color=club_default_color, shape="dot", size=30, x=x, y=y, physics=False, group="club")


# --- 監督ノード配置（年度に応じて色付け） ---
for node, data in G.nodes(data=True):
    if data["type"] != "manager":
        continue

    # 担当クラブの年度を安全に抽出
    years = []
    for club in G.neighbors(node):
        for y in G[node][club]["years"]:
            m = re.search(r"\d{4}", y)
            if m:
                years.append(int(m.group()))
    if not years:
        color = "#66CCFF"
    else:
        max_year = max(years)
        color = year_to_color(max_year)

    neighbours = list(G.neighbors(node))
    if len(neighbours) == 1:
        club = neighbours[0]
        cx, cy = club_positions[club]
        managers = [m for m in G.neighbors(club) if G.nodes[m]['type']=='manager' and len(list(G.neighbors(m)))==1]
        n = len(managers)
        idx = managers.index(node)
        r = 120   # 変更: クラブエンブレムと被りにくくするため半径を拡大
        angle = 2 * pi * idx / n
        mx = cx + r * cos(angle)
        my = cy + r * sin(angle)
        net.add_node(node, label="", title=node, color=color, shape="dot", size=12, x=mx, y=my, physics=False, group="manager")
    else:
        net.add_node(node, label="", title=node, color=color, shape="dot", size=12, physics=True, group="manager")

# --- エッジ追加 ---
for u, v, data in G.edges(data=True):
    try:
        years_sorted = []
        for y in data["years"]:
            m = re.search(r"\d{4}", y)
            if m:
                years_sorted.append(int(m.group()))
        years_sorted = sorted(years_sorted)
        net.add_edge(u, v, color="#BDB8B8", title=", ".join(map(str, years_sorted)))
    except Exception as e:
        print(f"エッジ追加でエラー: {u}-{v}, {e}")

# --- レイアウト設定 ---
net.set_options("""
var options = {
  "edges": {"color":{"inherit":false},"smooth":{"type":"dynamic"}},
  "nodes":{"font":{"size":14,"face":"arial"},"scaling":{"min":10,"max":30}},
  "interaction":{"hover":true,"tooltipDelay":100,"selectConnectedEdges":true},
  "physics":{"barnesHut":{"gravitationalConstant":-20000,"centralGravity":0.3,"springLength":180,"springConstant":0.04},"minVelocity":0.75}
}
""")

# 変更: net.show の代わりに HTML を書き出して JS を注入して選択時に強調する
html_path = "manager_club_network.html"
net.write_html(html_path)

# HTML に挿入するスクリプト（選択ノードとその隣接ノードのみを強調し、それ以外はフェードさせる）
injection = r"""
<script>
(function () {
  // 元プロパティを保持（色・枠・ラベル・サイズ・フォント）
  var originalNodeProps = {};
  var originalEdgeColors = {};

  nodes.get().forEach(function(n){
    originalNodeProps[n.id] = {
      color: n.color || null,
      borderWidth: n.borderWidth || null,
      label: (typeof n.label !== 'undefined') ? n.label : null,
      size: n.size || null,
      font: n.font || null,
      group: n.group || null,
      shape: n.shape || null,
      title: n.title || null
    };
  });
  edges.get().forEach(function(e){
    originalEdgeColors[e.id] = { color: e.color || null };
  });

  function fadeAllExcept(setIds, setEdges) {
    var updates = [];
    var nn = nodes.get();
    nn.forEach(function(n){
      var id = n.id;
      var orig = originalNodeProps[id] || {};
      var isIn = setIds.has(id) || setIds.has(String(id)) || setIds.has(Number(id));
      if (isIn) {
        // 選択対象 or 隣接は元の色・ラベルを復元（ただしラベルはマネージャのみ復元）
        updates.push({
          id: id,
          color: orig.color !== null ? orig.color : undefined,
          borderWidth: orig.borderWidth !== null ? orig.borderWidth : undefined,
          label: (orig.group === 'manager') ? (orig.label || orig.title || String(id)) : (orig.label || undefined),
          size: orig.size !== null ? orig.size : undefined,
          font: orig.font !== null ? orig.font : undefined
        });
      } else {
        // 非選択は薄くする。マネージャはラベルを非表示・小さくする
        if (orig.group === 'manager') {
          updates.push({
            id: id,
            color: { background: 'rgba(180,180,180,0.12)', border: 'rgba(180,180,180,0.12)' },
            label: "",
            size: 10
          });
        } else {
          updates.push({
            id: id,
            color: { background: 'rgba(180,180,180,0.12)', border: 'rgba(180,180,180,0.12)' }
          });
        }
      }
    });
    nodes.update(updates);

    // エッジ更新
    var eupd = [];
    var ee = edges.get();
    ee.forEach(function(e){
      var id = e.id;
      var isIn = setEdges.has(id) || setEdges.has(String(id)) || setEdges.has(Number(id));
      if (isIn) {
        var orig = originalEdgeColors[id];
        eupd.push({ id: id, color: orig && orig.color ? orig.color : undefined });
      } else {
        eupd.push({ id: id, color: { color: 'rgba(180,180,180,0.12)' } });
      }
    });
    edges.update(eupd);
  }

  network.on('selectNode', function(params) {
    var sel = params.nodes || [];
    if (sel.length === 0) return;
    var setIds = new Set();
    var setEdges = new Set();
    sel.forEach(function(nid){
      setIds.add(nid);
      var neigh = network.getConnectedNodes(nid);
      neigh.forEach(function(x){ setIds.add(x); });
      var conEdges = network.getConnectedEdges(nid);
      conEdges.forEach(function(eid){ setEdges.add(eid); });
    });

    // まずフェード処理（復元もここで行う）
    fadeAllExcept(setIds, setEdges);

    // マネージャノードは選択時にさらにサイズアップしてラベルを表示
    var mgrUpdates = [];
    setIds.forEach(function(id){
      var n = nodes.get(id);
      var orig = originalNodeProps[id] || {};
      if (n && orig.group === 'manager') {
        var newSize = Math.max(orig.size || 12, 18); // 最低拡大サイズ
        mgrUpdates.push({ id: id, size: newSize, label: orig.label || orig.title || String(id) });
      }
    });
    if (mgrUpdates.length) nodes.update(mgrUpdates);
  });

  network.on('deselectNode', function(params) {
    // 全てフェード（元の非選択状態）に戻す
    fadeAllExcept(new Set(), new Set());
  });

  // ページ読み込み時に全体を薄くして、監督ラベルを非表示にしておく
  (function initialFade(){
    var allIds = new Set();
    nodes.get().forEach(function(n){ allIds.add(n.id); });
    // 全て薄める。ラベルはマネージャのみ非表示にするため fadeAllExcept を利用
    fadeAllExcept(new Set(), new Set());
  })();

})();
</script>
"""

# HTML に挿入
with open(html_path, 'r', encoding='utf-8') as f:
    html = f.read()
if '</body>' in html:
    html = html.replace('</body>', injection + '</body>')
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(html)

print("manager_club_network.html を出力しました。選択ノードと隣接ノードのみ強調表示されます。")

manager_club_network.html を出力しました。選択ノードと隣接ノードのみ強調表示されます。
